# Lab 06: AgentCore Gateway with Semantic Tool Search

## Overview

In this notebook, we integrate **AgentCore Gateway** with **semantic search** enabled. When you have many tools, semantic search finds only the relevant tools for each query - reducing latency and cost.

**The Problem:**
- Enterprise MCP servers can have hundreds or thousands of tools
- Loading all tools into every request increases latency and token usage
- Poor tool selection accuracy with too many tools

**The Solution:**
- Enable semantic search on AgentCore Gateway
- Use `x_amz_bedrock_agentcore_search` to find relevant tools per query
- Pass only relevant tools to the agent

**How Semantic Search Works:**
```
Query: "What's your return policy?"
         ↓
   Gateway Search (x_amz_bedrock_agentcore_search)
         ↓
   Returns: [get_return_policy]  ← Only 1 relevant tool
         ↓
   Agent uses minimal tool set
```

> **Note:** This workshop uses only 4 tools, so the benefit of semantic search is limited. In production scenarios with 50-100+ tools, semantic search significantly reduces context size and improves tool selection accuracy. For this demo, we take only the top-ranked tool to illustrate the concept.

**What you'll learn:**
- How to create a Gateway with semantic search enabled
- How to use `x_amz_bedrock_agentcore_search` to find relevant tools
- How this reduces latency compared to loading all tools

## Prerequisites

- Completed Labs 01-05

## Workshop Journey

```
01 Baseline → 02 Quick Wins → 03 Caching → 04 Routing → 05 Guardrails → [06 Gateway] → 07 Evaluations
                                                                             ↑
                                                                        You are here
```

## Step 1: Setup and Imports

In [ ]:
from __future__ import annotations

import base64
import json
import os
import uuid
from pathlib import Path

import requests
from dotenv import load_dotenv

load_dotenv(override=True)

import boto3
from bedrock_agentcore_starter_toolkit import Runtime
from mcp.client.streamable_http import streamable_http_client
from strands.tools.mcp import MCPClient

region = os.environ.get("AWS_REGION", os.environ.get("AWS_DEFAULT_REGION", "us-east-1"))
ssm_client = boto3.client("ssm", region_name=region)
gateway_client = boto3.client("bedrock-agentcore-control", region_name=region)
cognito_client = boto3.client("cognito-idp", region_name=region)
agentcore_runtime = Runtime()

print(f"Region: {region}")
print(f"Langfuse Host: {os.environ.get('LANGFUSE_BASE_URL', 'https://cloud.langfuse.com')}")

---

**What's Different in This Lab?**

In previous labs (v1-v5), tools were **Python functions defined locally** in `utils/tools.py` and loaded directly into the agent:

```python
# v1-v5: Tools loaded directly from local files
from utils.tools import get_return_policy, get_product_info, web_search
agent = Agent(tools=[get_return_policy, get_product_info, ...])
```

In this lab, tools are **registered with AgentCore Gateway** — a centralized MCP server that hosts tools and provides them to agents via the Model Context Protocol (MCP):

```python
# v6: Tools accessed via Gateway
mcp_client = MCPClient(gateway_url)
tools = mcp_client.search_tools(query)  # Semantic search finds relevant tools
agent = Agent(tools=tools)
```

This enables tool sharing across multiple agents and semantic search for relevant tools.

---

## Step 2: Helper Functions

In [2]:
def get_ssm_parameter(name):
    """Retrieve a parameter from SSM Parameter Store."""
    response = ssm_client.get_parameter(Name=name)
    return response["Parameter"]["Value"]


def put_ssm_parameter(name, value):
    """Store a parameter in SSM Parameter Store."""
    ssm_client.put_parameter(Name=name, Value=value, Type="String", Overwrite=True)


def get_cognito_token(client_id, client_secret, token_url, scope):
    """Get OAuth2 token using client_credentials flow."""
    auth = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()
    response = requests.post(
        token_url,
        headers={
            "Authorization": f"Basic {auth}",
            "Content-Type": "application/x-www-form-urlencoded",
        },
        data={"grant_type": "client_credentials", "scope": scope},
    )
    response.raise_for_status()
    return response.json()["access_token"]


print("Helper functions defined")

Helper functions defined


## Step 3: Get Cognito Configuration

AgentCore Gateway uses JWT authentication. We'll use Cognito to generate tokens.

In [3]:
# Get Cognito configuration from SSM
try:
    cognito_client_id = get_ssm_parameter("/app/customersupport/agentcore/client_id")
    cognito_pool_id = get_ssm_parameter("/app/customersupport/agentcore/pool_id")
    cognito_discovery_url = get_ssm_parameter("/app/customersupport/agentcore/cognito_discovery_url")
    cognito_token_url = get_ssm_parameter("/app/customersupport/agentcore/cognito_token_url")
    cognito_scope = get_ssm_parameter("/app/customersupport/agentcore/cognito_auth_scope")

    print("Cognito configuration loaded from SSM:")
    print(f"  Client ID: {cognito_client_id[:20]}...")
    print(f"  Pool ID: {cognito_pool_id}")
    print(f"  Discovery URL: {cognito_discovery_url}")
except Exception as e:
    print(f"Error loading Cognito config: {e}")
    print("Make sure you've deployed the Cognito stack: make deploy-cognito")

Cognito configuration loaded from SSM:
  Client ID: 3pb9lghsc9arkh99k2rd...
  Pool ID: us-east-1_pz5W7bbxC
  Discovery URL: https://cognito-idp.us-east-1.amazonaws.com/us-east-1_pz5W7bbxC/.well-known/openid-configuration


In [4]:
# Get client secret from Cognito
try:
    client_response = cognito_client.describe_user_pool_client(
        UserPoolId=cognito_pool_id,
        ClientId=cognito_client_id,
    )
    client_secret = client_response["UserPoolClient"]["ClientSecret"]
    print("Client secret retrieved successfully")
except Exception as e:
    print(f"Error getting client secret: {e}")
    client_secret = None

Client secret retrieved successfully


In [5]:
# Get bearer token for Gateway authentication
if client_secret:
    bearer_token = get_cognito_token(
        cognito_client_id,
        client_secret,
        cognito_token_url,
        cognito_scope,
    )
    print(f"Bearer token obtained: {bearer_token[:50]}...")
else:
    bearer_token = None
    print("No bearer token - Gateway auth will not work")

Bearer token obtained: eyJraWQiOiJOd3pnXC9QTk1hcGY1MkJ5ZlVHVnZkK2NISzNoTF...


---

**Why JWT Authentication?**

Unlike local tools that run in the same process as your agent, Gateway tools are accessed over HTTPS. The Gateway requires authentication to:

- **Control access** — Only authorized agents can use the tools
- **Enable multi-tenancy** — Different teams can have different tool access
- **Audit usage** — Track which agents are calling which tools

The bearer token we just obtained will be passed in the `Authorization` header when connecting to the Gateway.

---

## Step 4: Create AgentCore Gateway

Now we create the Gateway itself — a managed MCP server that will host our tools.

In [ ]:
import time


def wait_for_gateway_ready(gateway_id, timeout_seconds=120, poll_interval=5):
    """Poll gateway status until READY or timeout."""
    start_time = time.time()
    while time.time() - start_time < timeout_seconds:
        response = gateway_client.get_gateway(gatewayIdentifier=gateway_id)
        status = response.get("status")
        if status == "READY":
            return True
        if status == "FAILED":
            raise Exception(f"Gateway creation failed: {response.get('statusReasons', 'Unknown reason')}")
        print(f"  Gateway status: {status}, waiting...")
        time.sleep(poll_interval)
    raise Exception(f"Timeout waiting for gateway to become READY (current status: {status})")


gateway_name = "workshop-customer-support-gateway"

# Get Gateway IAM role from SSM
gateway_role_arn = get_ssm_parameter("/app/customersupport/agentcore/gateway_iam_role")
print(f"Gateway IAM Role: {gateway_role_arn}")

# Configure JWT authorizer
auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [cognito_client_id],
        "discoveryUrl": cognito_discovery_url,
    }
}

# Enable semantic search - this creates a vector store for tool discovery
search_config = {"mcp": {"searchType": "SEMANTIC", "supportedVersions": ["2025-03-26"]}}

try:
    # Create new gateway with semantic search enabled
    print(f"Creating gateway: {gateway_name}")
    create_response = gateway_client.create_gateway(
        name=gateway_name,
        roleArn=gateway_role_arn,
        protocolType="MCP",
        authorizerType="CUSTOM_JWT",
        authorizerConfiguration=auth_config,
        protocolConfiguration=search_config,  # Enable semantic search
        description="Customer Support Workshop Gateway with Semantic Search",
    )

    gateway_id = create_response["gatewayId"]
    gateway_url = create_response["gatewayUrl"]
    gateway_arn = create_response["gatewayArn"]

    # Store in SSM for later use
    put_ssm_parameter("/app/customersupport/agentcore/gateway_id", gateway_id)
    put_ssm_parameter("/app/customersupport/agentcore/gateway_url", gateway_url)
    put_ssm_parameter("/app/customersupport/agentcore/gateway_arn", gateway_arn)

    # Wait for gateway to be ready before proceeding
    print("Waiting for gateway to become ready...")
    wait_for_gateway_ready(gateway_id)

    print("Gateway created successfully with semantic search!")
    print(f"  ID: {gateway_id}")
    print(f"  URL: {gateway_url}")

except gateway_client.exceptions.ConflictException:
    # Gateway already exists, get existing (no wait needed - already ready)
    print(f"Gateway '{gateway_name}' already exists, retrieving...")
    gateway_id = get_ssm_parameter("/app/customersupport/agentcore/gateway_id")
    gateway_response = gateway_client.get_gateway(gatewayIdentifier=gateway_id)
    gateway_url = gateway_response["gatewayUrl"]
    gateway_arn = gateway_response["gatewayArn"]
    print("Using existing gateway:")
    print(f"  ID: {gateway_id}")
    print(f"  URL: {gateway_url}")

except Exception as e:
    print(f"Error creating gateway: {e}")
    gateway_id = None
    gateway_url = None

---

**What is AgentCore Gateway?**

AgentCore Gateway is a **managed MCP (Model Context Protocol) server** that:

1. **Hosts tools centrally** — Tools are registered once and shared across agents
2. **Provides semantic search** — The `protocolConfiguration` with `searchType: "SEMANTIC"` enables vector-based tool discovery
3. **Handles authentication** — JWT tokens control who can access which tools
4. **Exposes an MCP endpoint** — Agents connect via the Gateway URL using standard MCP protocol

The Gateway URL (`/mcp` endpoint) is what our agent will connect to instead of loading tools locally.

---

## Step 5: Register Tools with Gateway

Now we register our tools with the Gateway. Instead of Python functions, we define **tool schemas** that describe each tool's name, description, and parameters.

In [7]:
# Tool schemas for all customer support tools
# These descriptions are indexed for semantic search
tool_schema = [
    {
        "name": "get_return_policy",
        "description": "Get return policy information for a product category including return windows, conditions, refund timelines, and warranty coverage. Use for questions about returns, refunds, exchanges, or warranties.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "product_category": {
                    "type": "string",
                    "description": "Product category (smartphones, laptops, tablets, audio, accessories)",
                }
            },
            "required": ["product_category"],
        },
    },
    {
        "name": "get_product_info",
        "description": "Get detailed product specifications, pricing, and availability for electronics. Use for questions about product features, specs, prices, or comparisons.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "product_type": {
                    "type": "string",
                    "description": "Type of product (laptops, smartphones, tablets, audio, accessories)",
                }
            },
            "required": ["product_type"],
        },
    },
    {
        "name": "get_technical_support",
        "description": "Get troubleshooting help and technical support from the knowledge base. Use for technical issues, setup problems, device malfunctions, or maintenance questions.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "issue_description": {"type": "string", "description": "Description of the technical issue or problem"}
            },
            "required": ["issue_description"],
        },
    },
    {
        "name": "web_search",
        "description": "Search the web for current information, recent news, promotions, or updates. Use when the user asks about current events, latest news, or time-sensitive information.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "keywords": {"type": "string", "description": "Search query keywords"},
                "max_results": {"type": "integer", "description": "Maximum number of results to return (default: 5)"},
            },
            "required": ["keywords"],
        },
    },
]

print(f"Defined {len(tool_schema)} tool schemas for Gateway:")
for tool in tool_schema:
    print(f"  - {tool['name']}")

Defined 4 tool schemas for Gateway:
  - get_return_policy
  - get_product_info
  - get_technical_support
  - web_search


---

**From Python Functions to Tool Schemas**

In previous labs, tools were Python functions with docstrings:

```python
# v1-v5: Python function
def get_return_policy(product_category: str) -> str:
    """Get return policy for a product category."""
    ...
```

For Gateway registration, we define the **same tools as JSON schemas**. The schema includes:
- `name` — Tool identifier
- `description` — Used for semantic search indexing (be descriptive!)
- `inputSchema` — JSON Schema defining parameters

The actual tool implementation lives in a **Lambda function** (the "target") — the Gateway routes tool calls to this Lambda.

---

In [8]:
if gateway_id:
    # Get Lambda ARN from SSM
    lambda_arn = get_ssm_parameter("/app/customersupport/agentcore/lambda_arn")
    print(f"Lambda ARN: {lambda_arn}")

    # Configure Lambda target with all tools
    lambda_target_config = {
        "mcp": {
            "lambda": {
                "lambdaArn": lambda_arn,
                "toolSchema": {"inlinePayload": tool_schema},
            }
        }
    }

    credential_config = [{"credentialProviderType": "GATEWAY_IAM_ROLE"}]

    try:
        # Create gateway target
        target_response = gateway_client.create_gateway_target(
            gatewayIdentifier=gateway_id,
            name="customer-support-tools",
            description="Customer support tools with semantic search",
            targetConfiguration=lambda_target_config,
            credentialProviderConfigurations=credential_config,
        )
        print(f"Gateway target created: {target_response['targetId']}")
        print(f"Registered {len(tool_schema)} tools for semantic search")

    except gateway_client.exceptions.ConflictException:
        print("Gateway target already exists")

    except Exception as e:
        print(f"Error creating gateway target: {e}")
else:
    print("Skipping target creation - no gateway")

Lambda ARN: arn:aws:lambda:us-east-1:739907928487:function:prompt-workshop-infra-CustomerSupportLambda-cDF4wq8VDt8c
Gateway target already exists


---

**What is a Gateway Target?**

A **Gateway Target** connects tool schemas to their implementations:

```
Agent → Gateway → Target (Lambda) → Tool execution
```

- **Gateway** — Routes requests and handles auth
- **Target** — The backend that executes tools (Lambda, in our case)
- **Tool Schema** — Tells the Gateway what tools exist and their parameters

When an agent calls a tool via MCP, the Gateway:
1. Validates the request against the schema
2. Forwards it to the Lambda target
3. Returns the result to the agent

This decouples tool definitions from implementations — you can update Lambda code without changing schemas.

---

## Step 6: Connect to Gateway via MCP

The agent connects to the Gateway using the MCP (Model Context Protocol) client. When a query comes in, the agent calls `x_amz_bedrock_agentcore_search` to find relevant tools based on the query semantics.

---

**What is MCP (Model Context Protocol)?**

MCP is an open protocol for connecting AI models to external tools and data. Think of it as a standardized way for agents to discover and call tools.

The `MCPClient` connects to the Gateway's MCP endpoint and can:
- **List available tools** — `list_tools()`
- **Call tools** — `call_tool(name, arguments)`
- **Search tools** — `x_amz_bedrock_agentcore_search` (Gateway's semantic search)

By using MCP, our agent can work with any MCP-compatible tool server, not just this specific Gateway.

---

In [ ]:
if gateway_url and bearer_token:
    print(f"Gateway Endpoint: {gateway_url}")

    # Set up MCP client with authentication
    mcp_client = MCPClient(
        lambda: streamable_http_client(
            gateway_url,
            headers={"Authorization": f"Bearer {bearer_token}"},
        )
    )
    print("MCP client configured")
else:
    mcp_client = None
    print("MCP client not available - missing gateway URL or token")

## Step 7: Test Semantic Search Locally

Before deploying, let's test the semantic search tool directly via MCP.

In [10]:
import time

# Wait for Gateway vector store to index tools
print("Waiting for Gateway to index tools...")
time.sleep(10)

# Test semantic search directly
if mcp_client:
    with mcp_client:
        # Search for tools related to return policy
        search_result = mcp_client.call_tool_sync(
            tool_use_id=str(uuid.uuid4()),
            name="x_amz_bedrock_agentcore_search",
            arguments={"query": "return policy for laptops"},
        )

        # Extract tools from structuredContent
        tools_found = search_result["structuredContent"]["tools"]
        print("Search query: 'return policy for laptops'")
        print(f"Tools found: {len(tools_found)}")
        for tool in tools_found:
            print(f"  - {tool['name']}")
else:
    print("MCP client not available")

Waiting for Gateway to index tools...
Search query: 'return policy for laptops'
Tools found: 4
  - customer-support-tools___get_return_policy
  - customer-support-tools___get_product_info
  - customer-support-tools___get_technical_support
  - customer-support-tools___web_search


---

**How Semantic Search Works**

The `x_amz_bedrock_agentcore_search` tool is a **built-in Gateway capability** that:

1. Takes a natural language query
2. Converts it to a vector embedding
3. Searches the indexed tool descriptions
4. Returns tools ranked by semantic relevance

The first tool in the results is the **most relevant** to the query. With only 4 tools, the Gateway returns all of them — but notice the **ranking order** reflects semantic similarity to the query.

---

## Step 8: Deploy the v6 Agent

Now let's deploy the agent that uses semantic search to AgentCore Runtime.

In [11]:
# Review the v6 agent code
agent_file = Path("agents/v6_gateway.py")
print(agent_file.read_text())

"""
V6 Gateway Agent - AgentCore Gateway with semantic tool search.
"""

import base64
import os
import uuid

import requests
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from mcp.client.streamable_http import streamablehttp_client
from mcp.types import Tool as MCPTool
from strands import Agent
from strands.models import BedrockModel
from strands.telemetry import StrandsTelemetry
from strands.tools.mcp import MCPClient
from strands.tools.mcp.mcp_client import MCPAgentTool
from strands.types.content import SystemContentBlock

from utils.agent_config import (
    MODEL_HAIKU,
    MODEL_SONNET,
    SYSTEM_PROMPT_TEXT,
    classify_query_complexity,
    setup_langfuse_telemetry,
)

setup_langfuse_telemetry()

app = BedrockAgentCoreApp()

GATEWAY_URL = os.environ.get("GATEWAY_URL")
GUARDRAIL_ID = os.environ.get("GUARDRAIL_ID")

# Cognito credentials (injected as env vars at deploy time)
COGNITO_CLIENT_ID = os.environ.get("COGNITO_CLIENT_ID")
COGNITO_CLIENT_SECRET = os.environ.g

---

**Key Differences from v4/v5**

Notice what's **no longer** in the agent code:

```python
# v4/v5 had local tool imports:
from utils.tools import get_return_policy, get_product_info, web_search
```

Instead, the v6 agent:

1. **Connects to Gateway** via `MCPClient` with JWT authentication
2. **Searches for relevant tools** using `x_amz_bedrock_agentcore_search`
3. **Converts search results** to `MCPAgentTool` objects the agent can use
4. **Creates agent with dynamic tools** — different queries may load different tools

The tools themselves still exist (in Lambda), but the agent discovers them at runtime via the Gateway rather than importing them at startup.

---

## Step 9: Configure and Deploy

The v6 agent uses `x_amz_bedrock_agentcore_search` to find relevant tools per query, reducing context size.

In [ ]:
agent_name = "customer_support_v6_gateway"
agent_file = str(Path("agents/v6_gateway.py").absolute())
requirements_file = str(Path("requirements-for-agentcore.txt").absolute())

# Clean up any existing build files from previous labs
for f in ["Dockerfile", ".dockerignore", ".bedrock_agentcore.yaml"]:
    p = Path(f)
    if p.exists():
        p.unlink()
        print(f"Removed existing: {f}")

print(f"Configuring agent: {agent_name}")
agentcore_runtime.configure(
    entrypoint=agent_file,
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file=requirements_file,
    region=region,
    agent_name=agent_name,
)

In [13]:
# Modify Dockerfile for Langfuse
import re

dockerfile_path = Path("Dockerfile")
if dockerfile_path.exists():
    content = dockerfile_path.read_text()
    if "opentelemetry-instrument" in content:
        content = re.sub(
            r'CMD \["opentelemetry-instrument", "python", "-m", "([^"]+)"\]', r'CMD ["python", "-m", "\1"]', content
        )
        dockerfile_path.write_text(content)
        print("Dockerfile modified for Langfuse")
    else:
        print("Dockerfile already configured")

Dockerfile modified for Langfuse


In [14]:
env_vars = {
    "LANGFUSE_BASE_URL": os.environ.get("LANGFUSE_BASE_URL"),
    "LANGFUSE_PUBLIC_KEY": os.environ.get("LANGFUSE_PUBLIC_KEY"),
    "LANGFUSE_SECRET_KEY": os.environ.get("LANGFUSE_SECRET_KEY"),
    "GATEWAY_URL": gateway_url or "",
    "GUARDRAIL_ID": os.environ.get("GUARDRAIL_ID", ""),
    # Cognito credentials for Gateway authentication
    "COGNITO_CLIENT_ID": cognito_client_id,
    "COGNITO_CLIENT_SECRET": client_secret,
    "COGNITO_TOKEN_URL": cognito_token_url,
    "COGNITO_SCOPE": cognito_scope,
    "PYTHONUNBUFFERED": "1",
}

print("Deploying to AgentCore Runtime...")
print(f"  Gateway URL: {gateway_url}")
print(f"  Cognito credentials: {'configured' if client_secret else 'missing'}")
launch_result = agentcore_runtime.launch(env_vars=env_vars, auto_update_on_conflict=True)
agent_arn = launch_result.agent_arn
print(f"Agent deployed: {agent_arn}")

🚀 Launching Bedrock AgentCore (cloud mode - RECOMMENDED)...
   • Deploy Python code directly to runtime
   • No Docker required (DEFAULT behavior)
   • Production-ready deployment

💡 Deployment options:
   • runtime.launch()                → Cloud (current)
   • runtime.launch(local=True)      → Local development
Memory disabled - skipping memory creation
Starting CodeBuild ARM64 deployment for agent 'customer_support_v6_gateway' to account 739907928487 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: customer_support_v6_gateway


Deploying to AgentCore Runtime...
  Gateway URL: https://workshop-customer-support-gateway-rgkzl6gn83.gateway.bedrock-agentcore.us-east-1.amazonaws.com/mcp
  Cognito credentials: configured


ECR repository available: 739907928487.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-customer_support_v6_gateway
Getting or creating execution role for agent: customer_support_v6_gateway
Using AWS region: us-east-1, account ID: 739907928487
Role name: AmazonBedrockAgentCoreSDKRuntime-us-east-1-74b3989be9


✅ Reusing existing ECR repository: 739907928487.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-customer_support_v6_gateway


✅ Reusing existing execution role: arn:aws:iam::739907928487:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-74b3989be9
Execution role available: arn:aws:iam::739907928487:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-74b3989be9
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: customer_support_v6_gateway
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-74b3989be9
Reusing existing CodeBuild execution role: arn:aws:iam::739907928487:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-74b3989be9
Using dockerignore.template with 46 patterns for zip filtering
Uploaded source to S3: customer_support_v6_gateway/source.zip
Updated CodeBuild project: bedrock-agentcore-customer_support_v6_gateway-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.4s
🔄 PROVISIONING started (total: 2s)
✅ PROVISIONING completed in 6.8s
🔄 DOWNLOAD_SO

Agent deployed: arn:aws:bedrock-agentcore:us-east-1:739907928487:runtime/customer_support_v6_gateway-gtZd6HBB4d


In [ ]:
# Save the agent ARN for later use
agent_arn = launch_result.agent_arn
print(f"Agent ARN: {agent_arn}")

## Step 10: Define Invocation Helper

In [15]:
data_client = boto3.client("bedrock-agentcore", region_name=region)


def invoke_agent(prompt):
    """Invoke the deployed agent via AgentCore API."""
    response = data_client.invoke_agent_runtime(
        agentRuntimeArn=agent_arn,
        runtimeSessionId=str(uuid.uuid4()),
        payload=json.dumps({"prompt": prompt}).encode(),
    )
    return json.loads(response["response"].read().decode("utf-8"))

In [16]:
# Import Langfuse metrics helper
from utils.langfuse_metrics import (
    clear_metrics,
    collect_metric,
    get_latest_trace_metrics,
    print_metrics,
    print_metrics_table,
)

# Clear any previously collected metrics
clear_metrics()
print("Metrics helper ready")

Metrics helper ready


## Step 11: Test the Agent

Run the same test prompts as previous labs to compare results.

> **Note:** For this demo, we take only the **top-ranked tool** from semantic search results. With only 4 tools, the Gateway returns all of them as potentially relevant. In production with 100+ tools, the search would naturally filter to a smaller relevant subset.

In [17]:
# Same test prompts as v2, v3, v4 notebooks for consistent comparison
TEST_PROMPTS = [
    ("Return Policy", "What is your return policy for laptops?"),
    ("Product Info", "Tell me about your smartphone options"),
    ("Technical Support", "My laptop won't turn on, can you help me troubleshoot?"),
    ("Multi-part Question", "I want to buy a laptop. What are the specs and what's the return policy?"),
    ("General Question", "Hello! What can you help me with today?"),
]

# Run all tests
print("=" * 70)
print("V6 GATEWAY WITH SEMANTIC TOOL SEARCH")
print("=" * 70)
print("Each query uses x_amz_bedrock_agentcore_search to find relevant tools.")
print("Watch the 'tools_loaded' count - semantic search finds relevant tools.")
print("=" * 70)

for test_name, prompt in TEST_PROMPTS:
    print("\n" + "-" * 70)
    print(f"Test: {test_name}")
    print(f"Query: {prompt}")
    print("-" * 70)

    response = invoke_agent(prompt)

    # Show tools_loaded from semantic search
    tools_loaded = response.get("tools_loaded", "N/A")
    print(f"Tools Loaded: {tools_loaded} (via semantic search)")
    print(f"Response: {str(response.get('response', response))[:200]}...")

    # Fetch and collect metrics
    metrics = get_latest_trace_metrics(
        agent_name="customer-support-v6-gateway",
        wait_seconds=5,
        max_retries=5,
        timeout_seconds=120,
    )
    print_metrics(metrics, test_name)
    collect_metric(metrics, test_name)

V6 GATEWAY WITH SEMANTIC TOOL SEARCH
Each query uses x_amz_bedrock_agentcore_search to find relevant tools.
Watch the 'tools_loaded' count - semantic search finds relevant tools.

----------------------------------------------------------------------
Test: Return Policy
Query: What is your return policy for laptops?
----------------------------------------------------------------------
Tools Loaded: 1 (via semantic search)
Response: - **answer:** Our laptop return policy is designed to give you peace of mind with your purchase:

  - **Return Window:** 30 days from the date of purchase
  - **Condition Requirements:** Laptops must ...

                    LANGFUSE METRICS
  Test:          Return Policy
  Trace ID:      27c98dea9590bf05357a8d083513f580
------------------------------------------------------------
  Latency:       9.14s
  Cost:          $0.007493
  Input tokens:  5,483
  Output tokens: 402
  Total tokens:  5,885
------------------------------------------------------------
 

Observe that for each prompt, the agent first does a semantic search on the most relevant tool, i.e.  "Tools Loaded: 1 (via semantic search)", and the proceed to use that tool.

In [18]:
# Print summary table
print_metrics_table()


                                  METRICS SUMMARY
               Test Latency    Cost Input Output Cache Read Tokens Cache Write Tokens
      Return Policy   9.14s $0.0075 5,483    402                 0                  0
       Product Info  11.19s $0.0081   757    360             1,383              1,383
  Technical Support  12.31s $0.0102   779    468             2,766                  0
Multi-part Question   7.79s $0.0081 5,635    500                 0                  0
   General Question   3.47s $0.0033   332    123             1,383                  0
---------------------------------------------------------------------------------------------------------
  TOTALS: Latency(avg): 8.78s | Cost: $0.0372 | Input: 12,986 | Output: 1,853
          Cache Read Tokens: 5,532 | Cache Write Tokens: 1,383



,Test,Latency,Cost,Input,Output,Cache Read Tokens,Cache Write Tokens
0,Return Policy,9.14s,$0.0075,"5,483",402,0,0
1,Product Info,11.19s,$0.0081,757,360,"1,383","1,383"
2,Technical Support,12.31s,$0.0102,779,468,"2,766",0
3,Multi-part Question,7.79s,$0.0081,"5,635",500,0,0
4,General Question,3.47s,$0.0033,332,123,"1,383",0


## Step 12: Compare with v4 (Routing)

Enter your metrics from Lab 04 (v4 routing) to compare with v6 gateway results.

In [19]:
from utils.langfuse_metrics import print_comparison

# ============================================================
# INPUT YOUR V4 METRICS FROM LAB 04 HERE
# (Copy the totals from your v4 metrics table)
# ============================================================
v4_total_cost = 0.0393  # e.g., 0.0393
v4_avg_latency = 8.88  # e.g., 8.88 (seconds)
v4_total_input_tokens = 14008  # e.g., 14008
v4_total_output_tokens = 1895  # e.g., 1895

# Print comparison (current metrics auto-calculated from collected)
print_comparison(
    prev_name="v4 (Routing)",
    curr_name="v6 (Gateway)",
    prev_cost=v4_total_cost,
    prev_latency=v4_avg_latency,
    prev_input_tokens=v4_total_input_tokens,
    prev_output_tokens=v4_total_output_tokens,
)

  V4 (ROUTING) vs V6 (GATEWAY) COMPARISON
Metric                     v4 (Routing)       v6 (Gateway)       Change
----------------------------------------------------------------------
Total Cost           $           0.0393 $           0.0372        -5.5%
Avg Latency (s)                    8.88               8.78        -1.1%
Input Tokens                     14,008             12,986        -7.3%
Output Tokens                     1,895              1,853        -2.2%

Result: 5.5% cost reduction, 1.1% latency improvement


### Key Observation

While the cost and latency changes may appear modest, notice the **significant reduction in input tokens**. This is the direct result of semantic search loading only 1 relevant tool per query instead of all 4.

**Why this matters at scale:**
- With 4 tools, the token savings are noticeable but not dramatic
- With 50-100+ tools, loading all tool definitions would consume thousands of tokens per request
- Semantic search keeps input tokens constant regardless of how many tools are registered in the Gateway

The Gateway pattern becomes essential when your MCP server has many tools — without semantic search, context windows would quickly fill up with tool definitions alone.

---

> **Note on Gateway Pricing:** The cost comparison above reflects only LLM token costs. AgentCore Gateway has its own consumption-based pricing (per Search API call, per InvokeTool call, and per tool indexed). These costs are omitted for simplicity but should be factored into production cost estimates. Gateway pricing is designed for scale — the value increases as you add more tools and agents.

## Summary

In this lab, we integrated **AgentCore Gateway** with **semantic tool search** — a fundamentally different approach to how agents access tools.

### The Shift: From Local to Centralized Tools

In previous labs, tools were Python functions bundled with the agent code. The agent imported them at startup and always had access to all tools, regardless of whether they were needed for a particular query.

With AgentCore Gateway, tools live in a centralized MCP server. The agent connects to the Gateway at runtime, searches for relevant tools using natural language, and loads only what it needs. This decouples tool management from agent deployment.

### What We Built

1. **Created an AgentCore Gateway** with semantic search enabled, providing a centralized MCP endpoint for tool access
2. **Registered tool schemas** that describe our four customer support tools, with descriptions optimized for semantic matching
3. **Connected a Lambda target** that executes the actual tool logic when called through the Gateway
4. **Deployed a v6 agent** that authenticates with the Gateway, searches for relevant tools per query, and dynamically loads only what's needed

### When AgentCore Gateway Makes Sense

The Gateway pattern is most valuable when:

- **You have many tools** — With 50-100+ tools, semantic search prevents context window bloat
- **Multiple agents share tools** — Register once, use from any agent
- **Teams manage tools independently** — Tool implementations can be updated without redeploying agents
- **You need access control** — JWT authentication enables fine-grained permissions

For our 4-tool workshop example, the benefits are modest. In enterprise scenarios with hundreds of tools across multiple teams, the Gateway becomes essential infrastructure.

---

**Next:** In Lab 07, we'll run comprehensive evaluations to compare all agent versions and measure the cumulative impact of our optimizations.

## Cleanup (Optional)

In [ ]:
# Uncomment to delete resources created in this lab
# Note: Cognito resources are managed by infrastructure stack

# # Delete the agent
# agent_id = agent_arn.split("/")[-1]
# control_client.delete_agent_runtime(agentRuntimeId=agent_id)
# print(f"Deleted agent: {agent_name}")

# # Delete ECR repository
# ecr_client = boto3.client("ecr", region_name=region)
# ecr_repo_name = f"bedrock-agentcore-{agent_name}"
# try:
#     ecr_client.delete_repository(repositoryName=ecr_repo_name, force=True)
#     print(f"Deleted ECR repository: {ecr_repo_name}")
# except ecr_client.exceptions.RepositoryNotFoundException:
#     print(f"ECR repository not found: {ecr_repo_name}")

# # Delete the Gateway target (must delete before Gateway)
# if gateway_id:
#     try:
#         targets = gateway_client.list_gateway_targets(gatewayIdentifier=gateway_id)
#         for target in targets.get("targets", []):
#             gateway_client.delete_gateway_target(
#                 gatewayIdentifier=gateway_id,
#                 targetIdentifier=target["targetId"]
#             )
#             print(f"Deleted Gateway target: {target['targetId']}")
#     except Exception as e:
#         print(f"Error deleting targets: {e}")

# # Delete the Gateway
# if gateway_id:
#     gateway_client.delete_gateway(gatewayIdentifier=gateway_id)
#     print(f"Deleted Gateway: {gateway_id}")